In [1]:
import os
from tqdm import tqdm
import re
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset, random_split, RandomSampler, Dataset
import torch
import torch.nn.functional as F
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from random import randrange
import math
import random
import re
import argparse

In [2]:
def loadFileData(folder, path):
    input_file = os.path.join(folder, path)
    with open(input_file, "r") as f:
        data = f.read()
    return data

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
friends_folderPath = "/project/Friends_Script_Txt"
friends_e1 = loadFileData(friends_folderPath,"0101.txt")
friends_e1

'The One Where Monica Gets a New Roommate (The Pilot-The Uncut Version)\n\n\n\n\n* * *\n\n\n\nWritten by: Marta Kauffman & David Crane\n\nTranscribed by: guineapig\n\nAdditional transcribing by: Eric Aasen\n\n(Note: The previously unseen parts of this episode are shown in blue text.)\n\n\n\n* * *\n\n\n\n[Scene: Central Perk, Chandler, Joey, Phoebe, and Monica are there.]\n\nMonica: There\'s nothing to tell! He\'s just some guy I work with!\n\nJoey: C\'mon, you\'re going out with the guy! There\'s gotta be something wrong with him!\n\nChandler: All right Joey, be nice. So does he have a hump? A hump and a hairpiece?\n\nPhoebe: Wait, does he eat chalk?\n\n(They all stare, bemused.)\n\nPhoebe: Just, \'cause, I don\'t want her to go through what I went through with Carl- oh!\n\nMonica: Okay, everybody relax. This is not even a date. It\'s just two people going out to dinner and- not having sex.\n\nChandler: Sounds like a date to me.\n\n[Time Lapse]\n\nChandler: Alright, so I\'m back in hig

In [4]:
friends_allText = ''
friendsFiles = os.listdir(friends_folderPath)
friendsFiles.sort()
for epPath in tqdm(friendsFiles):
    episodeText = loadFileData(friends_folderPath,epPath)
    episodeText = re.sub(r'^((.|\n)+?)(\[Scene)',r'\3',episodeText)
    episodeText = re.sub(r'\[Scene:',r'[',episodeText)
    friends_allText = friends_allText + '' + episodeText


100%|██████████| 228/228 [00:00<00:00, 600.91it/s]


In [5]:
def regexCleaning(text):
    text = re.sub(r"[cC]an\'t",r"cannot",text)
    text = re.sub(r"[cC]\'mon",r"come on",text)
    text = re.sub(r"(\\'re([^a-zA-Z])|\'re([^a-zA-Z]))",r" are\3",text)
    text = re.sub(r"(n\\'t([^a-zA-Z])|n\'t([^a-zA-Z]))",r" not\3",text)
    text = re.sub(r"(\\'m([^a-zA-Z])|\'m([^a-zA-Z]))",r" am\3",text)
    text = re.sub(r"(\\'ll([^a-zA-Z])|\'ll([^a-zA-Z]))",r" will\3",text)
    text = re.sub(r"\(",r"[",text)
    text = re.sub(r"\)",r"]",text)
    text = re.sub(r"\[",r"\n[",text)
    text = re.sub(r"\]",r"]\n",text)
    text = re.sub(r"(\\'ve([^a-zA-Z])|\'ve([^a-zA-Z]))",r" have\3",text)
    text = re.sub(r"\{((.|\n)+?)\}",'',text)
    text = re.sub(r"(\n)+",r"\1",text)
    text = re.sub(r"([a-z]):",r"\1 :",text)
    #text = re.sub(r"([a-zA-z])(\W)",r"\1 \2",text)
    
    return text

In [6]:
friends_allText_cleaned = regexCleaning(friends_allText)

In [7]:
nlines_friends = re.split('(\n)+',friends_allText_cleaned)
print(len(nlines_friends))
nlines_friends = [x for x in nlines_friends if x not in ['\n','',' ']]
wordsLen_friends = [len(line.split()) for line in nlines_friends]
print(max([x for x in wordsLen_friends]))
#[x[0] for x in wordsLen_friends].index(158)
print('Number of lines in friends: {} and average count of words per line: {}'.format(len(nlines_friends),round(np.median(wordsLen_friends))))

201149
158
Number of lines in friends: 100277 and average count of words per line: 7.0


In [8]:
nlines_friends = [x for x in nlines_friends if not ((len(x.split())==2) and (len(re.findall(r'(Ross|Rachel|Joey|Chandler|Phoebe|Monica) :',x))>=1))]
len(nlines_friends)

95348

In [9]:
nlines_friends[:5]

['[ Central Perk, Chandler, Joey, Phoebe, and Monica are there.]',
 "Monica : There's nothing to tell! He's just some guy I work with!",
 "Joey : come on, you are going out with the guy! There's gotta be something wrong with him!",
 'Chandler : All right Joey, be nice. So does he have a hump? A hump and a hairpiece?',
 'Phoebe : Wait, does he eat chalk?']

In [10]:
import importlib
if importlib.util.find_spec('allennlp') is None:
    # Check that allennlp is installed
    !pip install allennlp==0.9.0

if importlib.util.find_spec('flair') is None:
    !pip install flair==0.10
    
import flair
from flair.data import Sentence
from flair.embeddings import WordEmbeddings, ELMoEmbeddings, TransformerWordEmbeddings, TransformerDocumentEmbeddings

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [26]:
from transformers import BartForConditionalGeneration, BartTokenizer, BartConfig, AdamW

model_friends = BartForConditionalGeneration.from_pretrained("facebook/bart-base")
tok = BartTokenizer.from_pretrained("facebook/bart-base")

In [12]:
def noise_sentence(sentence_, percent_words, replacement_token = "<mask>"):
    '''
    The function takes a sentence as an input and masks part of the sentence with the <mask>
    token
    sentence - the sentence to noise
    percent_words - the percent of words to remove
    '''
    
    sentence = sentence_.split(' ')
    
    num_words = math.ceil(len(sentence) * percent_words)
    
    # We exclude the first word from getting masked because that is usually the name 
    # of a character and adds a lot of value to the current and the next sentences
    sampleTokens = set(np.arange(1, np.maximum(2, len(sentence)-1)))
    wordsToNoise = random.sample(sampleTokens, num_words)
    for index in wordsToNoise:
        sentence[index] = '<mask>'


    
    # Remove redundant spaces
    sentence = re.sub(r'( )+', ' ', ' '.join(sentence))
    
    # Combine concurrent <mask> tokens into a single token; this just does two rounds of this; more could be done
    sentence = re.sub(r'(<mask>)+', "<mask>", sentence)
    return sentence

In [13]:
def prepareData(data):
    train, validate = np.split(data.sample(frac=1), [int(.85*len(data))])
    return train, validate

In [14]:
def encode_sentences(tokenizer, source_sentences, target_sentences, max_length=128, return_tensors="pt"):
    ''' Takes a sentence as an input and tokenizes the sub words 
        tokenizer - the BART tokenizer; source and target sentences are the source and target sentences
    '''

    input_ids = []
    target_ids = []
    attention_masks = []
    tokenized_sentences = {}

    for sentence in tqdm(source_sentences):
        encoded_dict = tokenizer(
              sentence,
              max_length=max_length,
              padding="max_length",
              truncation=True,
              return_tensors=return_tensors,
              add_prefix_space = True
          )

        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
        
    input_ids = torch.cat(input_ids, dim = 0)
    attention_masks = torch.cat(attention_masks, dim = 0)

    for sentence in tqdm(target_sentences):
        encoded_dict = tokenizer(
              sentence,
              max_length=max_length,
              padding="max_length",
              truncation=True,
              return_tensors=return_tensors,
              add_prefix_space = True
          )
        target_ids.append(encoded_dict['input_ids'])

    target_ids = torch.cat(target_ids, dim = 0)

    batch = {
        "input_ids": input_ids,
        "attention_mask": attention_masks,
        "labels": target_ids,
    }

    return batch

In [15]:
def setupData(tokenizer,data):
    train = encode_sentences(tokenizer, data['Source'], data['target'])
    return train

In [16]:
def shift_tokens_right(input_ids, pad_token_id):
    """ Shift the input tokens for decoder to right, i.e. it removes the last non pad token and
    adds a sentence beginning token at the start
        This is taken directly from modeling_bart.py
    """
    prev_output_tokens = input_ids.clone()
    index_of_eos = (input_ids.ne(pad_token_id).sum(dim=1) - 1).unsqueeze(-1)
    prev_output_tokens[:, 0] = input_ids.gather(1, index_of_eos).squeeze()
    prev_output_tokens[:, 1:] = input_ids[:, :-1]
    return prev_output_tokens

In [17]:
featureWords, targetWords = [],[]
nlines_friends
for i in range(len(nlines_friends)-2):

    featureWord = nlines_friends[i]
#     if len(feature_word.split())>2:
#         feature_word = noise_sentence(feature_word, 0.25)
    targetWord = nlines_friends[i+1]
    featureWords.append(featureWord)
    targetWords.append(targetWord)

data = pd.DataFrame({"Source":featureWords,"target":targetWords})

train, val = prepareData(data)

print(data.head())

                                              Source  \
0  [ Central Perk, Chandler, Joey, Phoebe, and Mo...   
1  Monica : There's nothing to tell! He's just so...   
2  Joey : come on, you are going out with the guy...   
3  Chandler : All right Joey, be nice. So does he...   
4                  Phoebe : Wait, does he eat chalk?   

                                              target  
0  Monica : There's nothing to tell! He's just so...  
1  Joey : come on, you are going out with the guy...  
2  Chandler : All right Joey, be nice. So does he...  
3                  Phoebe : Wait, does he eat chalk?  
4                         [They all stare, bemused.]  


In [18]:
# Approach 2 (Not used in the end / Went ahead with Approach 1 instead)
'''
feature_words, target_words = [],[]
sequenceLen= 7
words_friendsE1 = friends_allText_cleaned.split()
for i in tqdm(range(len(words_friendsE1)-sequenceLen)):

    feature_word = words_friendsE1[i:i+sequenceLen]
    if len(feature_word)>2:
        feature_word = noise_sentence(feature_word, 0.25)
    target_word = words_friendsE1[i+sequenceLen]
    feature_words.append(feature_word)
    target_words.append(target_word)
    
    
data = pd.DataFrame({"Source":feature_words,"target":target_words})

train, val, test = prepareData(data)

print(data.head())
'''

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'\nfeature_words, target_words = [],[]\nsequenceLen= 7\nwords_friendsE1 = friends_allText_cleaned.split()\nfor i in tqdm(range(len(words_friendsE1)-sequenceLen)):\n\n    feature_word = words_friendsE1[i:i+sequenceLen]\n    if len(feature_word)>2:\n        feature_word = noise_sentence(feature_word, 0.25)\n    target_word = words_friendsE1[i+sequenceLen]\n    feature_words.append(feature_word)\n    target_words.append(target_word)\n    \n    \ndata = pd.DataFrame({"Source":feature_words,"target":target_words})\n\ntrain, val, test = prepareData(data)\n\nprint(data.head())\n'

In [19]:
train_data = setupData(tok,train)
train_dataset = TensorDataset(train_data['input_ids'], train_data['attention_mask'],train_data['labels'])
train_data = DataLoader(train_dataset, sampler = RandomSampler(train_dataset), batch_size = 16, num_workers = 8)

100%|██████████| 81044/81044 [00:25<00:00, 3167.57it/s]
/opt/anaconda/envs/Python3/lib/python3.8/site-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [20]:
val_data = setupData(tok,val)
val_dataset = TensorDataset(val_data['input_ids'],val_data['attention_mask'],val_data['labels'])
val_data = DataLoader(val_dataset, sampler = RandomSampler(val_dataset), batch_size = 16, num_workers=8, shuffle = False)

100%|██████████| 14302/14302 [00:04<00:00, 3166.63it/s]


In [21]:
# test_data = setup(tok,test)
# test_dataset = TensorDataset(test_data['input_ids'],test_data['attention_mask'],test_data['labels'])
# test_data = DataLoader(test_dataset, sampler = RandomSampler(test_dataset), batch_size = 16)

In [27]:
class LitModel(pl.LightningModule):
    # Instantiate the model
    def __init__(self, learning_rate, tokenizer, model, hparams):
        super().__init__()
        self.tokenizer = tokenizer
        self.model = model
        self.learning_rate = learning_rate
        self.hparams.update(hparams)

        if self.hparams["freeze_encoder"]:
            freeze_params(self.model.get_encoder())

        if self.hparams["freeze_embeds"]:
            self.freeze_embeds()

    def freeze_embeds(self):
        '''adapted from finetune.py '''
        freeze_params(self.model.model.shared)
        for d in [self.model.model.encoder, self.model.model.decoder]:
            freeze_params(d.embed_positions)
            freeze_params(d.embed_tokens)

    # Doing a forward pass
    def forward(self, input_ids, **kwargs):
        return self.model(input_ids, **kwargs)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr = self.learning_rate)
        return optimizer

    def training_step(self, batch, batch_idx):
        srcIds, srcMask = batch[0], batch[1]
        tgtIds = batch[2]
        # Shift the input tokens for the decoder right
        decoder_input_ids = shift_tokens_right(tgtIds, tok.pad_token_id)

        # Run the model
        outputs = self(srcIds, attention_mask=srcMask, decoder_input_ids=decoder_input_ids, use_cache=False)
        lm_logits = outputs[0]
        # Defining the loss function
        ce_loss_fct = torch.nn.CrossEntropyLoss(ignore_index=self.tokenizer.pad_token_id)
        # Calculate the loss
        loss = ce_loss_fct(lm_logits.view(-1, lm_logits.shape[-1]), tgtIds.view(-1))

        return {'loss':loss}

    def validation_step(self, batch, batch_idx):

        srcIdssrc_ids, srcMask = batch[0], batch[1]
        tgtIds = batch[2]

        decoder_input_ids = shift_tokens_right(tgtIds, tok.pad_token_id)

        # Run the model and get the logits
        outputs = self(srcIdssrc_ids, attention_mask=srcMask, decoder_input_ids=decoder_input_ids, use_cache=False)
        lm_logits = outputs[0]

        ce_loss_fct = torch.nn.CrossEntropyLoss(ignore_index=self.tokenizer.pad_token_id)
        val_loss = ce_loss_fct(lm_logits.view(-1, lm_logits.shape[-1]), tgtIds.view(-1))
        print("val loss", val_loss)
        return {'loss': val_loss}

    # Method that generates text using the BartForConditionalGeneration's generate() method
    def generate_text(self, text, eval_beams, early_stopping = True, max_len = 128):
        ''' Function to generate text '''
        generated_ids = self.model.generate(
            text["input_ids"],
            attention_mask=text["attention_mask"],
            use_cache=True,
            decoder_start_token_id = self.tokenizer.pad_token_id,
            num_beams= eval_beams,
            do_sample=True,
            top_k=50,
            top_p=0.7,
            temperature=1.5,
            max_length = max_len,
            early_stopping = early_stopping
        )
        return [self.tokenizer.decode(w, skip_special_tokens=True, clean_up_tokenization_spaces=True) for w in generated_ids]
def freeze_params(model):
    ''' Function that takes a model as input (or part of a model) and freezes the layers for faster training
      adapted from finetune.py '''
    for layer in model.parameters():
        layer.requires_grad = False



In [28]:
hparams = {}

hparams["freeze_encoder"] = True
hparams["freeze_embeds"] = True
hparams["eval_beams"] = 5


model_friends = LitModel(learning_rate = 0.00002, tokenizer = tok, model = model_friends, hparams = hparams)

In [29]:
checkpoint = ModelCheckpoint(filepath='checkpoint_files_2/')
trainer = pl.Trainer(gpus = 1,
                     max_epochs = 1,
                     min_epochs = 1,
                     auto_lr_find = False,
                     checkpoint_callback = checkpoint,
                     progress_bar_refresh_rate = 5)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


In [30]:
trainer.fit(model_friends,train_data,val_data)


  | Name  | Type                         | Params
-------------------------------------------------------
0 | model | BartForConditionalGeneration | 139 M 


Validation sanity check: 0it [00:00, ?it/s]

val loss tensor(6.1070, device='cuda:0')
val loss tensor(5.6108, device='cuda:0')


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

val loss tensor(2.8441, device='cuda:0')
val loss tensor(2.4747, device='cuda:0')
val loss tensor(2.8294, device='cuda:0')
val loss tensor(2.8676, device='cuda:0')
val loss tensor(2.7307, device='cuda:0')
val loss tensor(2.2572, device='cuda:0')
val loss tensor(2.6720, device='cuda:0')
val loss tensor(2.6960, device='cuda:0')
val loss tensor(2.6641, device='cuda:0')
val loss tensor(2.6212, device='cuda:0')
val loss tensor(2.5271, device='cuda:0')
val loss tensor(2.8753, device='cuda:0')
val loss tensor(2.5973, device='cuda:0')
val loss tensor(2.7077, device='cuda:0')
val loss tensor(2.3587, device='cuda:0')
val loss tensor(2.2485, device='cuda:0')
val loss tensor(2.5185, device='cuda:0')
val loss tensor(2.4838, device='cuda:0')
val loss tensor(2.4498, device='cuda:0')
val loss tensor(2.6984, device='cuda:0')
val loss tensor(2.6474, device='cuda:0')
val loss tensor(2.1059, device='cuda:0')
val loss tensor(2.6232, device='cuda:0')
val loss tensor(2.2042, device='cuda:0')
val loss tensor(

val loss tensor(2.5778, device='cuda:0')
val loss tensor(2.4019, device='cuda:0')
val loss tensor(2.4891, device='cuda:0')
val loss tensor(3.1013, device='cuda:0')
val loss tensor(2.3770, device='cuda:0')
val loss tensor(2.3524, device='cuda:0')
val loss tensor(2.8011, device='cuda:0')
val loss tensor(2.7726, device='cuda:0')
val loss tensor(2.2909, device='cuda:0')
val loss tensor(2.6180, device='cuda:0')
val loss tensor(2.1179, device='cuda:0')
val loss tensor(2.5140, device='cuda:0')
val loss tensor(2.6211, device='cuda:0')
val loss tensor(2.7519, device='cuda:0')
val loss tensor(2.7454, device='cuda:0')
val loss tensor(2.7904, device='cuda:0')
val loss tensor(2.5489, device='cuda:0')
val loss tensor(2.9055, device='cuda:0')
val loss tensor(2.6595, device='cuda:0')
val loss tensor(2.5323, device='cuda:0')
val loss tensor(2.6501, device='cuda:0')
val loss tensor(2.1355, device='cuda:0')
val loss tensor(2.1171, device='cuda:0')
val loss tensor(2.8216, device='cuda:0')
val loss tensor(

val loss tensor(2.6542, device='cuda:0')
val loss tensor(2.4313, device='cuda:0')
val loss tensor(2.5721, device='cuda:0')
val loss tensor(1.9361, device='cuda:0')
val loss tensor(2.8597, device='cuda:0')
val loss tensor(2.6641, device='cuda:0')
val loss tensor(2.3152, device='cuda:0')
val loss tensor(2.5730, device='cuda:0')
val loss tensor(2.0997, device='cuda:0')
val loss tensor(2.5474, device='cuda:0')
val loss tensor(2.4619, device='cuda:0')
val loss tensor(2.7702, device='cuda:0')
val loss tensor(2.8748, device='cuda:0')
val loss tensor(3.0702, device='cuda:0')
val loss tensor(3.1615, device='cuda:0')
val loss tensor(2.0692, device='cuda:0')
val loss tensor(2.4911, device='cuda:0')
val loss tensor(2.1294, device='cuda:0')
val loss tensor(2.3517, device='cuda:0')
val loss tensor(2.4558, device='cuda:0')
val loss tensor(3.0404, device='cuda:0')
val loss tensor(2.5349, device='cuda:0')
val loss tensor(2.7319, device='cuda:0')
val loss tensor(2.9351, device='cuda:0')
val loss tensor(

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:25: RuntimeWarning: The metric you returned None must be a `torch.Tensor` instance, checkpoint not saved HINT: what is the value of val_loss in validation_epoch_end()?
  warnings.warn(*args, **kwargs)
/opt/anaconda/envs/Python3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:25: RuntimeWarning: Can save best model only with val_loss available, skipping.
  warnings.warn(*args, **kwargs)


1

In [72]:
def generate_text_friends(model, text, eval_beams, early_stopping = True, max_len = 128):
    ''' Function to generate text '''
    generated_ids = model.model.generate(
         text["input_ids"],
        attention_mask=text["attention_mask"],
        use_cache=True,
        decoder_start_token_id = tok.pad_token_id,
        num_beams= eval_beams,
        do_sample=True,
        top_k=50,
        top_p=0.9,
        temperature=1.5,
        max_length = max_len,
        output_scores=True,
        return_dict_in_generate=True,
        early_stopping = early_stopping
    )

    return [tok.decode(w, skip_special_tokens=True, clean_up_tokenization_spaces=True) for w in generated_ids['sequences']], generated_ids['sequences_scores']

def generate_script_friends(seed_line, num_lines, model_, multiple_lines = False, max_line_history = 5):
  # Put the model on eval mode
    model_.to(torch.device('cpu'))
    model_.eval()
    script_text = []
    script_text.append(seed_line)
    prompt_line_tokens = tok(seed_line, max_length = 128, return_tensors = "pt", truncation = True)
    # Loop through the number of lines generating a new line based on the old

    line = [seed_line]
    for i in range(num_lines):
        # Print out the new line
        print(line[0].strip())
        script_text.append(line[0])
        line, score = generate_text_friends(model_,prompt_line_tokens, eval_beams = 4)
        # This deals with an artefact in the training data that I had an issue cleaning
        # This allows the model to generate a new line conditioned on more than one line
        if multiple_lines:
            start_line = np.maximum(0, i - max_line_history)
            end_line = i
            prompt_line = ' '.join(script_text[start_line:end_line]) # Going to end_line is fine because it is non-inclusive
        else:
            prompt_line = script_text[i]
        prompt_line_tokens = tok(prompt_line, max_length = 128, return_tensors = "pt", truncation = True)

    #return ' '.join(script_text)

In [32]:
new_script_friends = generate_script_friends(seed_line = "Ross: We were on a break!", num_lines = 14, model_ = model_friends,
                           multiple_lines = False, max_line_history = 4)

Ross: We were on a break!
Phoebe : Oh, I am so sorry, but I really, really want to get back together with you guys. I mean, I thought you were the one who had a baby, but what do you mean?
Phoebe : Oh yeah, we are on a break.
Phoebe : Oh, yeah! I’m sorry, you’re right.
Rachel : Well, I think I have a problem with that.
Monica : I know! It's just I don’t think you know, I mean, I just feel like it’s just not right.
Monica : You’re right! Yeah, you are right.
Ross : No, no, you can’t. I have to get out of here.
Phoebe : Oh yeah. I know, that’s what I feel.
Ross : No, I’m not. I don’t want to do anything to make you feel better.
[He starts to leave, but Rachel stops him.]
[Joey gets up and walks away.]
Joey : Well, I think you’re just gonna have to get used to it.
Rachel : What?! What are you doing?


In [34]:
got_e1 = loadFileData('',"/project/Game_of_Thrones/Data/season1/e1.txt")
got_e1

'[First scene opens with three Rangers riding through a tunnel, leaving the Wall, and going into the woods. (Eerie music in background) One Ranger splits off and finds a campsite full of mutilated bodies, including a child hanging from a tree branch. A birds-eye view shows the bodies arranged in a shield-like pattern. The Ranger rides back to the other two.]\nWAYMAR ROYCE: What d’you expect? They’re savages. One lot steals a goat from another lot and before you know it, they’re ripping each other to pieces.\nWILL: I’ve never seen wildlings do a thing like this. I’ve never seen a thing like this, not ever in my life.\nWAYMAR ROYCE: How close did you get?\nWILL: Close as any man would.\nGARED: We should head back to the wall.\nROYCE: Do the dead frighten you?\nGARED: Our orders were to track the wildlings. We tracked them. They won’t trouble us no more.\nROYCE: You don’t think he’ll ask us how they died? Get back on your horse.\n[GARED grumbles.]\nWILL: Whatever did it to them could do i

In [35]:
got_folders = os.listdir("/project/Game_of_Thrones/Data")
got_folders = [x for x in got_folders if not 'txt' in x]
got_folders.sort()
got_allText = ''

for ix1, folder in tqdm(enumerate(got_folders)):
    gotFiles = os.listdir("/project/Game_of_Thrones/Data/"+str(folder))
    for ix2, epPath in enumerate(['e1.txt', 'e2.txt', 'e3.txt', 'e4.txt', 'e5.txt', 'e6.txt', 'e7.txt', 'e8.txt', 'e9.txt', 'e10.txt', 'e11.txt' ]):
        if epPath in gotFiles:
            episodeText = loadFileData("/project/Game_of_Thrones/Data/"+str(folder),epPath)
            if not (ix1 == 0 and ix2 == 0):
                got_allText = got_allText + '\n\nEnd\n\n\n\n\n\n' + episodeText
            else:
                got_allText = got_allText + ''+ episodeText





0it [00:00, ?it/s]
2it [00:00, 14.43it/s]
4it [00:00, 13.21it/s]
6it [00:00, 12.59it/s]
8it [00:00, 13.85it/s]


In [36]:
got_allText = regexCleaning(got_allText)
got_allText

'\n[First scene opens with three Rangers riding through a tunnel, leaving the Wall, and going into the woods. \n[Eerie music in background]\n One Ranger splits off and finds a campsite full of mutilated bodies, including a child hanging from a tree branch. A birds-eye view shows the bodies arranged in a shield-like pattern. The Ranger rides back to the other two.]\nWAYMAR ROYCE: What d’you expect? They’re savages. One lot steals a goat from another lot and before you know it, they’re ripping each other to pieces.\nWILL: I’ve never seen wildlings do a thing like this. I’ve never seen a thing like this, not ever in my life.\nWAYMAR ROYCE: How close did you get?\nWILL: Close as any man would.\nGARED: We should head back to the wall.\nROYCE: Do the dead frighten you?\nGARED: Our orders were to track the wildlings. We tracked them. They won’t trouble us no more.\nROYCE: You don’t think he’ll ask us how they died? Get back on your horse.\n[GARED grumbles.]\nWILL: Whatever did it to them coul

In [37]:
nlines_got = re.split('(\n)+',got_allText)
nlines_got = [x for x in nlines_got if x not in ['\n','',' ']]
wordsLen_got = [len(line.split()) for line in nlines_got]
print(max([x for x in wordsLen_got]))
print([x for x in wordsLen_got].index(361))

print('Number of lines in GOT: {} and average count of words per line: {}'.format(len(nlines_got),round(np.mean(wordsLen_got))))

361
29550
Number of lines in GOT: 35743 and average count of words per line: 13.0


In [38]:
feature_words_got, target_words_got = [],[]
nlines_got
for i in range(len(nlines_got)-2):

    feature_word = nlines_got[i]
#     if len(feature_word.split())>2:
#         feature_word = noise_sentence(feature_word, 0.25)
    target_word = nlines_got[i+1]
    feature_words_got.append(feature_word)
    target_words_got.append(target_word)

data_got = pd.DataFrame({"Source":feature_words_got,"target":target_words_got})

train_got, val_got = prepareData(data_got)

print(data_got.head())

                                              Source  \
0  [First scene opens with three Rangers riding t...   
1                        [Eerie music in background]   
2   One Ranger splits off and finds a campsite fu...   
3  WAYMAR ROYCE: What d’you expect? They’re savag...   
4  WILL: I’ve never seen wildlings do a thing lik...   

                                              target  
0                        [Eerie music in background]  
1   One Ranger splits off and finds a campsite fu...  
2  WAYMAR ROYCE: What d’you expect? They’re savag...  
3  WILL: I’ve never seen wildlings do a thing lik...  
4               WAYMAR ROYCE: How close did you get?  


In [39]:
train_data_got = setupData(tok,train_got)
train_dataset_got = TensorDataset(train_data_got['input_ids'], train_data_got['attention_mask'],train_data_got['labels'])
train_data_got = DataLoader(train_dataset_got, sampler = RandomSampler(train_dataset_got), batch_size = 16, num_workers = 8)

val_data_got = setupData(tok,val_got)
val_dataset_got = TensorDataset(val_data_got['input_ids'],val_data_got['attention_mask'],val_data_got['labels'])
val_data_got = DataLoader(val_dataset_got, sampler = RandomSampler(val_dataset_got), batch_size = 16, num_workers=8, shuffle = False)

# test_data_got = setup(tok,test_got)
# test_dataset_got = TensorDataset(test_data_got['input_ids'],test_data_got['attention_mask'],test_data_got['labels'])
# test_data_got = DataLoader(test_dataset_got, sampler = RandomSampler(test_dataset_got), batch_size = 16)

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)

100%|██████████| 30379/30379 [00:11<00:00, 2605.17it/s]

 17%|█▋        | 5151/30379 [00:01<00:09, 2802.26it/s]


100%|██████████| 30379/30379 [00:10<00:00, 2777.72it/s]
/opt/anaconda/envs/Python3/lib/python3.8/site-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(

100%|██████████| 5362/5362 [00:01<00:00, 2742.40it/s]

100%|██████████| 5362/5362 [00:01<00:00, 2882.87it/s]


In [40]:
model_got = BartForConditionalGeneration.from_pretrained("facebook/bart-base")
tok_got = BartTokenizer.from_pretrained("facebook/bart-base")

In [41]:
hparams_got = {}

hparams_got["freeze_encoder"] = True
hparams_got["freeze_embeds"] = True
hparams_got["eval_beams"] = 5


model_got = LitModel(learning_rate = 0.00002, tokenizer = tok_got, model = model_got, hparams = hparams_got)

In [42]:
trainer.fit(model_got,train_data_got,val_data_got)

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:25: RuntimeWarning: You have defined a `val_dataloader()` and have defined a `validation_step()`, you may also want to define `validation_epoch_end()` for accumulating stats.
  warnings.warn(*args, **kwargs)

  | Name  | Type                         | Params
-------------------------------------------------------
0 | model | BartForConditionalGeneration | 139 M 
/opt/anaconda/envs/Python3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: Your val_dataloader has `shuffle=True`, it is best practice to turn this off for validation and test dataloaders.
  warnings.warn(*args, **kwargs)


Validation sanity check: 0it [00:00, ?it/s]

val loss tensor(6.6927, device='cuda:0')
val loss tensor(5.7218, device='cuda:0')


Training: 5065it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

val loss tensor(2.8114, device='cuda:0')
val loss tensor(3.0361, device='cuda:0')
val loss tensor(2.5043, device='cuda:0')
val loss tensor(2.6823, device='cuda:0')
val loss tensor(2.4472, device='cuda:0')
val loss tensor(3.0098, device='cuda:0')
val loss tensor(2.7018, device='cuda:0')
val loss tensor(3.0388, device='cuda:0')
val loss tensor(2.9251, device='cuda:0')
val loss tensor(2.5628, device='cuda:0')
val loss tensor(2.8431, device='cuda:0')
val loss tensor(2.9611, device='cuda:0')
val loss tensor(2.9716, device='cuda:0')
val loss tensor(3.0564, device='cuda:0')
val loss tensor(2.5376, device='cuda:0')
val loss tensor(2.5955, device='cuda:0')
val loss tensor(2.5804, device='cuda:0')
val loss tensor(2.9499, device='cuda:0')
val loss tensor(2.9328, device='cuda:0')
val loss tensor(2.8069, device='cuda:0')
val loss tensor(2.9232, device='cuda:0')
val loss tensor(2.7755, device='cuda:0')
val loss tensor(2.5421, device='cuda:0')
val loss tensor(2.8434, device='cuda:0')
val loss tensor(

val loss tensor(2.7547, device='cuda:0')
val loss tensor(2.7751, device='cuda:0')
val loss tensor(2.7893, device='cuda:0')
val loss tensor(2.3948, device='cuda:0')
val loss tensor(2.7708, device='cuda:0')
val loss tensor(2.8135, device='cuda:0')
val loss tensor(2.6141, device='cuda:0')
val loss tensor(2.9600, device='cuda:0')
val loss tensor(2.6643, device='cuda:0')
val loss tensor(2.5833, device='cuda:0')
val loss tensor(3.0068, device='cuda:0')
val loss tensor(2.9734, device='cuda:0')
val loss tensor(2.9162, device='cuda:0')
val loss tensor(2.8575, device='cuda:0')
val loss tensor(2.4521, device='cuda:0')
val loss tensor(2.4075, device='cuda:0')
val loss tensor(2.6271, device='cuda:0')
val loss tensor(2.5406, device='cuda:0')
val loss tensor(2.7681, device='cuda:0')
val loss tensor(3.0534, device='cuda:0')
val loss tensor(2.9032, device='cuda:0')
val loss tensor(2.9381, device='cuda:0')
val loss tensor(2.8038, device='cuda:0')
val loss tensor(3.0540, device='cuda:0')
val loss tensor(

1

In [62]:
def generate_text_got(model, text, eval_beams, early_stopping = True, max_len = 128):
    ''' Function to generate text '''
    generated_ids = model_got.model.generate(
        text["input_ids"],
        attention_mask=text["attention_mask"],
        use_cache=True,
        decoder_start_token_id = tok.pad_token_id,
        num_beams= eval_beams,
        do_sample=True,
        top_k=50,
        top_p=0.7,
        temperature=1.5,
        max_length = max_len,
        output_scores=True,
        return_dict_in_generate=True,
        early_stopping = early_stopping
    )

    return [tok.decode(w, skip_special_tokens=True, clean_up_tokenization_spaces=True) for w in generated_ids['sequences']], generated_ids['sequences_scores']



def generate_script_got(seed_line, num_lines, model_, multiple_lines = False, max_line_history = 5):
  # Put the model on eval mode
    model_.to(torch.device('cpu'))
    model_.eval()
    script_text = []
    script_text.append(seed_line)
    prompt_line_tokens = tok(seed_line, max_length = 128, return_tensors = "pt", truncation = True)
    # Loop through the number of lines generating a new line based on the old

    line = [seed_line]
    for i in range(num_lines):
        # Print out the new line
        print(line[0].strip())
        script_text.append(line[0])
        line, score = generate_text_got(model_,prompt_line_tokens, eval_beams = 4)
        # This deals with an artefact in the training data that I had an issue cleaning
        # This allows the model to generate a new line conditioned on more than one line
        if multiple_lines:
            start_line = np.maximum(0, i - max_line_history)
            end_line = i
            prompt_line = ' '.join(script_text[start_line:end_line]) # Going to end_line is fine because it is non-inclusive
        else:
            prompt_line = script_text[i]
        prompt_line_tokens = tok(prompt_line, max_length = 128, return_tensors = "pt", truncation = True)

In [44]:
new_script_got = generate_script_got(seed_line = "TYRION: A Lannister always pays his debt.", num_lines = 14, model_ = model_got,
                           multiple_lines = False, max_line_history = 4)
new_script_got

TYRION: A Lannister always pays his debt.
DAVOS: What kind of debt does a Lannister owe you?
TYRION: He’ll pay for his debt to you.
JAQEN: You will pay it, my lord.
JON: My debt. It's my debt. That's what I owe you.
JAIME: He’ll pay you for your sins.
JAQEN: You are a rich man. I do not want to live in the North.
CERSEI: It's a debt. I owe you. What's my debt?
BRIENNE: He’s not going to do it.
JAQEN looks up at the sky. There is a shadow on her face.
TYRION: That's my debt. And you owe me a debt.
BRIENNE: I would not do it. If you were not going to do it, you’d do it with your life.
JAQEN: What are you talking about?
DAENERYS: You owe me a debt.


In [73]:
def generate_script_combined_withScore(seed_line, num_lines, model_friends, model_got, multiple_lines = False, max_line_history = 5):
  # Put the model on eval mode
    model_friends.to(torch.device('cpu'))
    model_friends.eval()
    model_got.to(torch.device('cpu'))
    model_got.eval()
    script_text = []
    script_text.append(seed_line)
    prompt_line_tokens = tok(seed_line, max_length = 128, return_tensors = "pt", truncation = True)
    # Loop through the number of lines generating a new line based on the old

    line = [seed_line]
    for i in range(num_lines):
        # Print out the new line
        print(line[0].strip())
        script_text.append(line[0])
        line_friends, score_friends = generate_text_friends(model_friends,prompt_line_tokens, eval_beams = 4)
        #print(score_friends)
        line_got, score_got = generate_text_got(model_got,prompt_line_tokens, eval_beams = 4)
        #print(score_got)
        if score_friends<score_got: # since scores are always negative and we have to consider the absolute value
            line = line_friends
        else:
            line = line_got
        # This deals with an artefact in the training data that I had an issue cleaning
        # This allows the model to generate a new line conditioned on more than one line
        
        if multiple_lines:
            start_line = np.maximum(0, i - max_line_history)
            end_line = i
            prompt_line = ' '.join(script_text[start_line:end_line]) # Going to end_line is fine because it is non-inclusive
        else:
            prompt_line = script_text[i]
        prompt_line_tokens = tok(prompt_line, max_length = 128, return_tensors = "pt", truncation = True)

In [74]:
generate_script_combined_withScore(seed_line = "[Chandler and Tyrion walked into Central Perk and ordered a pint of ale.]", num_lines = 14, model_friends= model_friends, model_got = model_got,
                           multiple_lines = True, max_line_history = 6)


[Chandler and Tyrion walked into Central Perk and ordered a pint of ale.]
Chandler : So, how do we get this to work?
Phoebe : You’re so sweet.
CHANDLER: I’m here.
:   A pint of ale?
Tyrion Lannister :  I do. I do not know.
CERSEI: I’m sorry.  I do not mean to insult you, but we’re here to talk to you.
[Ross and Chandler walk back into Central Perk.]
JON:    A pint of ale?
Joffrey Baratheon :  I do not know.
[He tries to kiss Phoebe]
[They’re both shocked]
[to Phoebe]
[Listens]


In [44]:
def generate_script_combined_random(seed_line, num_lines, model_friends, model_got, multiple_lines = False, max_line_history = 5):
  # Put the model on eval mode
    model_friends.to(torch.device('cpu'))
    model_friends.eval()
    model_got.to(torch.device('cpu'))
    model_got.eval()
    script_text = []
    script_text.append(seed_line)
    prompt_line_tokens = tok(seed_line, max_length = 128, return_tensors = "pt", truncation = True)
    # Loop through the number of lines generating a new line based on the old

    line = [seed_line]
    for i in range(num_lines):
        # Print out the new line
        print(line[0].strip())
        script_text.append(line[0])
        line_friends, score_friends = generate_text_friends(model_friends,prompt_line_tokens, eval_beams = 4)
        line_got, score_got = generate_text_got(model_got,prompt_line_tokens, eval_beams = 4)
        pickDict = {0:line_friends,1:line_got}
        line = pickDict[randrange(2)]
        # This deals with an artefact in the training data that I had an issue cleaning
        # This allows the model to generate a new line conditioned on more than one line
        
        if multiple_lines:
            start_line = np.maximum(0, i - max_line_history)
            end_line = i
            prompt_line = ' '.join(script_text[start_line:end_line]) # Going to end_line is fine because it is non-inclusive
        else:
            prompt_line = script_text[i]
        prompt_line_tokens = tok(prompt_line, max_length = 128, return_tensors = "pt", truncation = True)

In [52]:
generate_script_combined_random(seed_line = "[Chandler and Tyrion walked into Central Perk and ordered a pint of ale.]", num_lines = 14, model_friends= model_friends, model_got = model_got,
                           multiple_lines = True, max_line_history = 4)


[Chandler and Tyrion walked into Central Perk and ordered a pint of ale.]
[ Central Perk, Joey and Rachel are sitting on the couch as Phoebe enters.]
[ Central Perk, Phoebe, Ross, and Chandler are there as Monica enters.]
Theon Greyjoy :  It's not ale, it's ale.  You are drinking it, my lord.
Theon Greyjoy :  The only ale I’ll ever have.
[to Phoebe]
[entering]
Tyrion Lannister :  I know.  It's not ale, it's ale.
[They go into the living room]
[She goes to the door to see Monica, and Phoebe enters.]
Tyrion Lannister :  You’re drinking it, your lord.  And it’s not ale.
[he walks away, and Phoebe turns to look at him.]
[To Phoebe]
[To Phoebe]
